In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data from http://www.spotrac.com/nba/contracts/
# Initial Analysis
current_contracts = 'NBA_current_contracts.csv'
df = pd.read_csv(current_contracts, encoding = "ISO-8859-1")
df.columns
df.shape
df.info()
df.describe()

#Column Value Count
column_value_count = [df[column_label].value_counts(dropna=False)
                     for column_label in df.columns]
print(column_value_count)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 10 columns):
PLAYER         560 non-null object
POS            560 non-null object
TEAM           560 non-null object
AGE            560 non-null int64
YRS            560 non-null int64
DOLLARS        560 non-null object
GUARANTEED     560 non-null object
% GTD          560 non-null object
AVG. SALARY    560 non-null object
FREE AGENT     560 non-null int64
dtypes: int64(3), object(7)
memory usage: 43.8+ KB
[Jaylen Johnson (2017-2017)         1
Ben McLemore (2017-2018)           1
Willie Reed (2017-2017)            1
Daniel Ochefu (2016-2018)          1
K.J. McDaniels (2017-2017)         1
Abdel Nader (2017-2019)            1
Andrew Harrison (2016-2018)        1
Tony Allen (2017-2017)             1
Malcom Delaney (2016-2017)         1
Tim Quarterman (2016-2017)         1
Shaun Livingston (2017-2019)       1
Shabazz Muhammad (2017-2018)       1
Kevin Love (2015-2019)             1
Jaylen Brown (2

In [40]:
# RENAME % GTD, AVG. SALARY and FREE AGENT to cleaner format
rename = {'% GTD':'PERCENT_GTD', 
          'AVG. SALARY':'AVG_SALARY',
          'FREE AGENT':'FREE_AGENT'}

df = df.rename(columns = rename)
print(df.columns)
# DOLLARS, GUARANTEED, %_GTD,  is type object(str) need to change to type int
# First examine each
print(df.DOLLARS.head())
print(df.GUARANTEED.head())
print(df.PERCENT_GTD.head())

Index(['PLAYER', 'POS', 'TEAM', 'AGE', 'YRS', 'DOLLARS', 'GUARANTEED',
       'PERCENT_GTD', 'AVG_SALARY', 'FREE_AGENT'],
      dtype='object')
0    $205,030,000.00
1    $201,158,790.00
2    $171,174,820.00
3    $169,344,000.00
4    $169,344,000.00
Name: DOLLARS, dtype: object
0    $205,030,000.00
1    $201,158,790.00
2    $171,174,820.00
3    $169,344,000.00
4    $169,344,000.00
Name: GUARANTEED, dtype: object
0    100.00%
1    100.00%
2    100.00%
3    100.00%
4    100.00%
Name: PERCENT_GTD, dtype: object


In [41]:
# DOLLARS from STRING to INT
df.DOLLARS = df.DOLLARS.str.replace('$', '')
df.DOLLARS = df.DOLLARS.str.replace(',', '')
df.DOLLARS = pd.to_numeric(df.DOLLARS)
df.DOLLARS.head()
# GUARENTEED from STRING TO INT
df.GUARANTEED = df.GUARANTEED.str.replace('$', '')
df.GUARANTEED = df.GUARANTEED.str.replace(',', '')
df.GUARANTEED = pd.to_numeric(df.GUARANTEED, errors='coerce')
df.GUARANTEED.head()
# PERCENT_GTD from STRING TO INT
df.PERCENT_GTD = df.PERCENT_GTD.str.replace('%', '')
df.PERCENT_GTD = pd.to_numeric(df.PERCENT_GTD, errors='coerce')

In [42]:
# Determine year the contract was signed
df.year_signed = df.FREE_AGENT - df.YRS
print(df.year_signed.head())
# Note some years are in the future (2018, 2019) due to contract extension
# For these  the cap is currently unknown so we will have to extrapolate
print(df.year_signed.max())
# Need to extrapolate cap up to 2021

0    2018
1    2017
2    2017
3    2019
4    2019
dtype: int64
2021


In [43]:
# Bring in Salary Cap by Year data
cap_by_year = 'NBA_cap_by_year.csv'
df_cap = pd.read_csv(cap_by_year)
print(df_cap.head(), df_cap.info())

# Clean df_cap column names to match other dataframes
rename = {'Year':'YEAR',
          'Cap': 'CAP'}
df_cap = df_cap.rename(columns = rename)
      
# Clean YEAR column to 1st year 
# Contracts are signed is based off the start of the season cap
df_cap.YEAR = df_cap.YEAR.str.replace('-\d\d', '')

# Clean CAP from object to float
df_cap.CAP = df_cap.CAP.str.replace('$', '')
df_cap.CAP = df_cap.CAP.str.replace(',', '')
df_cap.CAP = pd.to_numeric(df_cap.CAP)
print(df_cap.head(), df_cap.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
Year    40 non-null object
Cap     40 non-null object
dtypes: object(2)
memory usage: 720.0+ bytes
      Year            Cap
0  1984-85  $3,600,000.00
1  1985-86  $4,233,000.00
2  1986-87  $4,945,000.00
3  1987-88  $6,164,000.00
4  1988-89  $7,232,000.00 None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
YEAR    40 non-null object
CAP     40 non-null float64
dtypes: float64(1), object(1)
memory usage: 720.0+ bytes
   YEAR        CAP
0  1984  3600000.0
1  1985  4233000.0
2  1986  4945000.0
3  1987  6164000.0
4  1988  7232000.0 None


In [64]:
# Calculate for each player their current contract as a % of the cap space the year it was signed
df.SALARY_PERCENT_OF_CAP = [df.DOLLARS[i] / df_cap.loc[df_cap.YEAR == df.year_signed[i], 'CAP'] for i in df.index]
print(df.head())

NameError: name 'i' is not defined